<a href="https://colab.research.google.com/github/Farjana-Yesmin/Farjana-Yesmin/blob/main/Task2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the neural network architecture
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load and preprocess the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Using MNIST dataset as an example
dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Instantiate the model, loss function, and optimizer
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    train_losses = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    return train_losses

# Function to evaluate the model on the test set
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test set: {accuracy:.2f}%')
    return all_preds, all_labels

# Train the model
num_epochs = 10
train_losses = train_model(model, train_loader, criterion, optimizer, num_epochs)

# Visualize training loss over epochs
plt.figure()
plt.plot(range(1, num_epochs + 1), train_losses, marker='o')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Evaluate the model
predictions, labels = evaluate_model(model, test_loader)

# Visualization of test set performance using confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

conf_matrix = confusion_matrix(labels, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix on Test Set')
plt.show()

# Additional quantitative performance visualization
# e.g., plotting class-wise accuracy if required, or plotting precision-recall for multi-class problems

# Hyperparameter tuning comments:
# The learning rate, batch size, and number of epochs were chosen based on standard practices for MNIST.
# Further tuning could be achieved using a learning rate scheduler or experimenting with deeper network architectures.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

# Set image size and noise parameters
img_height, img_width = 64, 64
noise_factor = 0.5  # Adjust noise factor to determine "meaningful" amount of Gaussian noise

# Load and preprocess the dataset (using MNIST as a placeholder)
(x_train, _), (x_test, _) = mnist.load_data()
x_train = np.resize(x_train, (x_train.shape[0], img_height, img_width))
x_test = np.resize(x_test, (x_test.shape[0], img_height, img_width))
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Add Gaussian noise
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Build the Denoising Autoencoder
input_img = Input(shape=(img_height, img_width, 1))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=2)(x)

# Decoder
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=2)(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same', strides=2)(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Model definition
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

# Train the model
history = autoencoder.fit(x_train_noisy, x_train,
                          epochs=50,
                          batch_size=128,
                          shuffle=True,
                          validation_data=(x_test_noisy, x_test))

# Evaluate on test data
denoised_imgs = autoencoder.predict(x_test_noisy)

# Plotting original, noisy, and denoised images for comparison
n = 10  # Number of images to display
plt.figure(figsize=(20, 6))
for i in range(n):
    # Display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(img_height, img_width), cmap="gray")
    plt.title("Original")
    plt.axis("off")

    # Display noisy image
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(x_test_noisy[i].reshape(img_height, img_width), cmap="gray")
    plt.title("Noisy")
    plt.axis("off")

    # Display denoised image
    ax = plt.subplot(3, n, i + 1 + 2 * n)
    plt.imshow(denoised_imgs[i].reshape(img_height, img_width), cmap="gray")
    plt.title("Denoised")
    plt.axis("off")
plt.show()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 64, 64, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 64, 64, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 64, 64, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 64, 64, 1)           │             289 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 157,601 (615.63 KB)

 Trainable params: 157,601 (615.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
198/469 ━━━━━━━━━━━━━━━━━━━━ 13:48 3s/step - loss: 0.4213

Explanation of the Code
Dataset Preparation: This code uses the MNIST dataset as a placeholder for natural images of size 64x64. You can replace it with your own grayscale dataset if available.

Adding Gaussian Noise: Zero-mean Gaussian noise with a predefined factor (e.g., 0.5) is added to simulate noisy images.
Model Architecture: The autoencoder has a minimum of four encoding and decoding layers.

Training: The model is trained until the reconstructed (denoised) images are of high quality.

Evaluation: The original, noisy, and denoised images are displayed to assess the autoencoder's performance.


Observations

Denoising Quality: Evaluate the denoising quality by visually inspecting the denoised images compared to the noisy ones.

Effect of Noise Factor: Experiment with different noise factors to find a "meaningful" amount of noise.

Loss Curves: You can also plot the training and validation loss curves to observe the convergence and performance of the model.

In [ ]:
!pip install tensorflow matplotlib open3d
!git clone https://github.com/YoYo000/BlendedMVS


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Reshape, UpSampling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import open3d as o3d  # For 3D point cloud visualization


In [ ]:
# Placeholder function to load and preprocess data (replace with actual data loading)
def load_data(path, split_ratio=0.8):
    # Load images and 3D points from the BlendedMVS dataset directory
    # This is a placeholder for dataset loading
    # Split data into training and test sets based on `split_ratio`
    x_train, y_train, x_test, y_test = [], [], [], []
    return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

x_train, y_train, x_test, y_test = load_data('/content/BlendedMVS')


In [ ]:
input_img = Input(shape=(256, 256, 3))  # Replace with your image shape

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=2)(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', strides=2)(x)
x = Flatten()(x)
encoded = Dense(1024, activation='relu')(x)

# Decoder (producing 3D point coordinates)
x = Dense(64 * 64 * 64, activation='relu')(encoded)
x = Reshape((64, 64, 64))(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Model
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='mse')
model.summary()


In [ ]:
history = model.fit(x_train, y_train, epochs=50, batch_size=16, validation_data=(x_test, y_test))


In [ ]:
mse = model.evaluate(x_test, y_test)
print("Mean Squared Error on Test Set:", mse)


In [ ]:
def visualize_point_cloud(prediction):
    # Convert prediction to 3D point cloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(prediction.reshape(-1, 3))
    o3d.visualization.draw_geometries([point_cloud])

# Visualize a sample prediction
sample_image = x_test[0:1]
predicted_points = model.predict(sample_image)
visualize_point_cloud(predicted_points[0])


Observations

Quantitative Observations: Note the MSE (or other metric values) and check how they vary with different architecture or data splits.

Qualitative Observations: Examine the visualizations of 3D point clouds. Check if the point cloud aligns with the expected geometry of the object.
